# User Guide: Basic Non-Cartesian Reconstruction Pipeline

This tutorial guides you through setting up and running a simple end-to-end 2D non-Cartesian MRI reconstruction pipeline using the `reconlib` library. We will cover the essential steps from simulating k-space data to reconstructing an image using a NUFFT operator and an iterative solver.

**Steps Covered:**
1. Setup and necessary imports.
2. Generation of a phantom and a non-Cartesian (radial) k-space trajectory.
3. Simulation of k-space data, including adding noise.
4. Configuration and instantiation of the NUFFT (Non-Uniform Fast Fourier Transform) operator.
5. Performing image reconstruction using the Conjugate Gradient solver.
6. Visualizing the results.

## 1. Setup and Imports

In [ ]:
%matplotlib inline
import torch
import numpy as np
import matplotlib.pyplot as plt

# ReconLib imports
try:
    from reconlib.nufft import NUFFT2D
    from reconlib.solvers import conjugate_gradient_reconstruction
    # For data simulation - assuming functions from iternufft.py are suitable
    # These were identified during the creation of voronoi_recon_comparison.ipynb
    from iternufft import generate_phantom_2d, generate_radial_trajectory_2d
    print("Successfully imported reconlib components and simulation utilities.")
except ImportError as e:
    print(f"Error importing modules: {e}")
    print("Please ensure reconlib is installed and iternufft.py is in the Python path.")

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 2. Data Generation

First, we'll define the parameters for our simulation, generate a simple 2D phantom, create a radial k-space trajectory, and then simulate the k-space data by applying the forward NUFFT operation. Finally, we'll add some noise to make the reconstruction task more realistic.

In [ ]:
# --- Configuration ---
IMAGE_SIZE = 128       # Size of the image (NxN)
N_SPOKES = 128         # Number of radial spokes in k-space
N_SAMPLES_PER_SPOKE = int(IMAGE_SIZE * 1.5) # Number of samples along each spoke (oversampling factor of 1.5 along readout)
NOISE_STD_PERCENT = 0.02 # Noise level as a percentage of max k-space signal

# --- Generate Phantom ---
try:
    phantom_img = generate_phantom_2d(size=IMAGE_SIZE, device=device)
    phantom_complex = phantom_img.to(torch.complex64)
    print(f"Phantom generated with shape: {phantom_img.shape}")
except NameError:
    print("generate_phantom_2d not available. Using a placeholder.")
    phantom_complex = torch.zeros((IMAGE_SIZE, IMAGE_SIZE), dtype=torch.complex64, device=device)

plt.figure(figsize=(4, 4))
plt.imshow(phantom_complex.abs().cpu().numpy(), cmap='gray')
plt.title(f"Original Phantom ({IMAGE_SIZE}x{IMAGE_SIZE})")
plt.axis('off')
plt.show()

# --- Generate K-Space Trajectory ---
try:
    k_trajectory = generate_radial_trajectory_2d(
        num_spokes=N_SPOKES,
        samples_per_spoke=N_SAMPLES_PER_SPOKE,
        device=device
    )
    print(f"K-space trajectory generated. Shape: {k_trajectory.shape}")
except NameError:
    print("generate_radial_trajectory_2d not available. Using placeholder.")
    k_trajectory = torch.rand((N_SPOKES * N_SAMPLES_PER_SPOKE, 2), device=device) - 0.5

plt.figure(figsize=(4, 4))
plt.scatter(k_trajectory[:, 0].cpu().numpy(), k_trajectory[:, 1].cpu().numpy(), s=0.5)
plt.title(f"K-Space Trajectory")
plt.xlabel("kx"); plt.ylabel("ky")
plt.axis('square')
plt.show()

### Simulate K-Space Data and Add Noise

In [ ]:
# --- Simulate K-Space Data (Forward NUFFT) ---
# First, we need a NUFFT operator for the forward simulation.
# We'll use default parameters for NUFFT2D for simplicity here.
# These parameters should be suitable for the k_trajectory and phantom_complex defined earlier.
# No explicit density compensation is typically used for the forward model.

try:
    nufft_params_sim = {
        'image_shape': (IMAGE_SIZE, IMAGE_SIZE),
        'k_trajectory': k_trajectory,
        'oversamp_factor': (2.0, 2.0), 
        'kb_J': (4, 4),      
        'kb_alpha': (2.34 * 4, 2.34 * 4), 
        'Ld': (1024, 1024), 
        'kb_m': (0.0, 0.0),       
        'device': device
    }
    nufft_op_sim = NUFFT2D(**nufft_params_sim)
    print("NUFFT operator for simulation created.")

    k_space_data_clean = nufft_op_sim.forward(phantom_complex)
    print(f"Clean k-space data simulated. Shape: {k_space_data_clean.shape}")

    # --- Add Complex Gaussian Noise ---
    noise_std_val = NOISE_STD_PERCENT * torch.max(torch.abs(k_space_data_clean))
    
    # Generate noise for real and imaginary parts separately then combine
    noise_real_part = torch.randn_like(k_space_data_clean.real) * noise_std_val
    noise_imag_part = torch.randn_like(k_space_data_clean.imag) * noise_std_val
    complex_noise = torch.complex(noise_real_part, noise_imag_part).to(device)
    
    k_space_data_noisy = k_space_data_clean + complex_noise
    print(f"Noisy k-space data created. Shape: {k_space_data_noisy.shape}")

except NameError as e:
    print(f"A required variable (IMAGE_SIZE, k_trajectory, device, phantom_complex, NOISE_STD_PERCENT, NUFFT2D) is not defined: {e}")
    print("Please ensure the previous cells have been run.")
    k_space_data_noisy = torch.zeros((k_trajectory.shape[0] if 'k_trajectory' in globals() else 1), dtype=torch.complex64, device=device) # Placeholder
except Exception as e:
    print(f"Error during k-space simulation or noise addition: {e}")
    k_space_data_noisy = torch.zeros((k_trajectory.shape[0] if 'k_trajectory' in globals() else 1), dtype=torch.complex64, device=device) # Placeholder


# --- Visualize K-Space (Magnitude) ---
plt.figure(figsize=(5,5))
if 'k_space_data_noisy' in globals() and k_space_data_noisy.numel() > 0 :
    plt.scatter(k_trajectory[:,0].cpu(), k_trajectory[:,1].cpu(), 
                c=torch.log(torch.abs(k_space_data_noisy.cpu()) + 1e-9), # log scale for better viz
                s=1, cmap='viridis')
    plt.colorbar(label="log|k-space data|")
else:
    plt.text(0.5,0.5, "K-space data not available", ha='center', va='center')
plt.title("Simulated Noisy K-Space Data (Log Magnitude)")
plt.xlabel("kx"); plt.ylabel("ky")
plt.axis('square')
plt.show()

## 3. NUFFT Operator Setup

In [ ]:
# --- NUFFT Operator Setup for Reconstruction ---
# For reconstruction, we instantiate another NUFFT operator.
# This operator will be used by the solver (e.g., Conjugate Gradient).
# For this basic tutorial, we'll configure it similarly to the simulation NUFFT,
# without explicitly passing density_comp_weights. The conjugate_gradient_reconstruction
# solver itself will handle passing relevant parts of these nufft_kwargs to the
# NUFFT2D constructor. If density_comp_weights were to be used (e.g. Voronoi),
# they would be included in this nufft_kwargs dictionary.

print("\n--- Setting up NUFFT Operator for Reconstruction ---")
try:
    # These parameters are largely the same as for the simulation NUFFT operator.
    # The k_trajectory and image_shape must match the data we are reconstructing.
    nufft_recon_kwargs = {
        'oversamp_factor': (2.0, 2.0), 
        'kb_J': (4, 4),      
        'kb_alpha': (2.34 * 4, 2.34 * 4), 
        'Ld': (1024, 1024), 
        'kb_m': (0.0, 0.0)
        # device, image_shape, k_trajectory will be passed by the solver
        # or when NUFFT2D is directly instantiated with all args.
        # For clarity here, we are defining the 'kwargs' part that the solver would use.
        # No 'density_comp_weights' are specified here for the most basic case.
        # The solver (e.g. conjugate_gradient_reconstruction) will instantiate
        # NUFFT2D using these kwargs along with image_shape, k_trajectory, and device.
    }
    
    # We can also instantiate it directly for clarity or if we wanted to perform
    # a simple adjoint reconstruction manually (though solvers handle this).
    nufft_op_recon_direct_instance = NUFFT2D(
        image_shape=(IMAGE_SIZE, IMAGE_SIZE), # from previous cell
        k_trajectory=k_trajectory,          # from previous cell
        device=device,                      # from previous cell
        **nufft_recon_kwargs
    )
    print(f"NUFFT2D operator for reconstruction purposes configured (can be instantiated by solver or directly).")
    print(f"Using parameters: image_shape={(IMAGE_SIZE,IMAGE_SIZE)}, os_factor={nufft_recon_kwargs['oversamp_factor']}, J={nufft_recon_kwargs['kb_J']}")
    if hasattr(nufft_op_recon_direct_instance, 'density_comp_weights') and nufft_op_recon_direct_instance.density_comp_weights is not None:
        print(f"NUFFT op has density_comp_weights of shape: {nufft_op_recon_direct_instance.density_comp_weights.shape}")
    else:
        print("NUFFT op configured without explicit external density_comp_weights (will use internal default if any).")

except NameError as e:
    print(f"A required variable (IMAGE_SIZE, k_trajectory, device, NUFFT2D) is not defined: {e}")
    print("Please ensure the previous cells have been run.")
    nufft_recon_kwargs = {} # Placeholder
except Exception as e:
    print(f"Error setting up NUFFT reconstruction operator: {e}")
    nufft_recon_kwargs = {} # Placeholder

## 4. Image Reconstruction

## 5. Visualization and Conclusion